In [40]:
import pandas as pd
import numpy as np
import re

from urllib.parse import urlparse
import contractions
import string
import num2words

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer
from nltk.corpus import words, stopwords
from nltk.metrics.distance import jaccard_distance, edit_distance
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [41]:
train_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/tanfiona CausalNewsCorpus master data-V2/test_subtask1_text.csv')
# dev_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/tanfiona CausalNewsCorpus master data-V2/dev_subtask1.csv')
# print(train_df.shape)
# print(dev_df.shape)
# train_df = pd.concat([train_df, dev_df], ignore_index=True)
# print(train_df.shape)
# train_df.head()

In [42]:
print(train_df.isna().sum())
print(train_df.shape)
print(train_df.dtypes)
# print(train_df.label.value_counts())

index    0
text     0
dtype: int64
(352, 2)
index    object
text     object
dtype: object


In [43]:
abbreviations = {
    'n\'t':'not',
    'won\'t':'will not',
    'can\'t':'can not',
    'i\'m':'i am',
    'you\'re':'you are',
    'he\'s':'he is',
    'she\'s':'she is',
    'it\'s':'it is',
    'we\'re':'we are',
    'they\'re':'they are',
    'i\'ve':'i have',
    'you\'ve':'you have',
    'we\'ve':'we have',
    'NaMo':'Narendra Modi',
    'PMO':'Prime Minister Office',
    'PM':'Prime Minister',
    'BJP':'Bharatiya Janata Party',
    'INC':'Indian National Congress',
    'AAP':'Aam Aadmi Party',
    'J&K':'Jammu and Kashmir',
    'NHAI':'National Highways Authority of India',
    'CPI':'Communist Party of India',
    'CPI(M)':'Communist Party of India (Marxist)',
    'CPI-M':'Communist Party of India (Marxist)',
    'HQs':'Headquarters',
    'NGO':'Non Governmental Organization',
    'NGOs':'Non Governmental Organizations',
}


In [44]:
def removing_links(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    x = [word for word in tokens if not urlparse(word).scheme]
    text = ' '.join(x)
    return text

def contractions_handling(text):
    text = contractions.fix(text)
    return text

def replacing_abbr(text, dictry):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    x = [dictry[word] if word in dictry.keys() else word for word in tokens]
    text = ' '.join(x)
    return text

def removing_punctuations(text):
    text = text.translate(str.maketrans(' ', ' ', string.punctuation))
    text = text.lower()
    return text

def removing_stopwords(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    x = [word for word in tokens if word not in stopwords.words('english')]
    text = ' '.join(x)
    return text

def convert_num_words(text):
    re_results = re.findall('(\d+(st|nd|rd|th))', text)
    for entire_result, suffix in re_results:
        num = int(entire_result[:-len(suffix)])
        entire_result = " " + entire_result
        num_word = num2words.num2words(num, ordinal=True)
        text = text.replace(entire_result, " " + num_word)
    
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [num2words.num2words(word) if word.isdigit() else word for word in tokens]
    text = ' '.join(tokens)
    
    return text

def removing_char_less_3(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [word for word in tokens if len(word) >= 3]
    text = ' '.join(tokens)
    return text

In [45]:
train_df['text'] = train_df['text'].apply(lambda x: removing_links(x))
train_df['text'] = train_df['text'].apply(lambda x: contractions_handling(x))
# train_df['text'] = train_df['text'].apply(lambda x: replacing_abbr(x, abbreviations))
train_df['text'] = train_df['text'].apply(lambda x: removing_punctuations(x))
# train_df['text'] = train_df['text'].apply(lambda x: removing_stopwords(x))
train_df['text'] = train_df['text'].apply(lambda x: convert_num_words(x))
train_df['text'] = train_df['text'].apply(lambda x: removing_char_less_3(x))

train_df.head()

,index,text
0,test_01_A_0_271,more than twenty associate degree students mar...
1,test_01_A_1_215,more than ten people from the rights associati...
2,test_01_A_10_198,the organisation has been spearheading agitati...
3,test_01_A_100_277,not believe politicians that they will achieve...
4,test_01_A_101_152,for that matter haragopal himself took proacti...


In [46]:
train_df.to_csv('/Users/nitanshjain/Documents/Projects/CASE/subtask_1/subtask_1_data/test_subtask1_test_preprocessed_4.csv', index=False)

In [47]:
# 1 - no stopwords and removed few abbreviations
# 2 - with stopwords and removed few abbreviations
# 3 - no stopwords and didnt remove abbreviations
# 4 - with stopwords and didnt remove abbreviations